# Exploratory Data Analysis (EDA)

## Scope of this project
Get familiar with the _King County Housing Data_ and perform an **Exploratory Data Analysis** (EDA) with focus on the following particular requests by the stakeholder.

The stakeholder: <br>
Nicole Johnson, buyer, who seeks for a "Lively, central neighborhood, middle price range, right timing (within a year)"

---------------------------
# Table of Content
1) Import and first impression of the dataset
2) Initial Hypotheses about the Dataset
3) Explore and clean the dataset
    - Add/remove columns
    - Filtering
    - Testing the hypotheses
4) Recommendations for the stakeholder
-------------------------------


## 1) Import and first impression of the dataset

First of all, we will load the data into the workspace as a _dataframe object_ using **_pandas_** and display the main characteristics of it.

In [ ]:
# import the necessary libraries we need for your analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# import dataset
df = pd.read_csv('data/King_County_House_prices_dataset.csv', parse_dates=['date'])
df

In [ ]:
# So, what size does the dataset has?
print("\n", f"The dataset has {df.shape[0]} rows and {df.shape[1]} columns.", "\n")

# Now, let us take a view to the columns and their type:
df.info()

- For most of the columns the dtype looks reasonable except for _sqft_basement_ which is of type "object", but we expect it to be a "float" since the variable gives us the size of the basement in square feet.
- For _waterfront_ we see that it is a "float64" and not boolean as we might have thought (either the house has a waterfront or not). So, lets have a quick view:

In [ ]:
df.waterfront.unique()


We see, rather than TRUE/FALSE the column has already been one-hot encoded and contains 0/1 as well as nan (not a number).<br>
Speaking of nan, how many nan do we have in each column?

In [ ]:
df.isna().sum()

Except for _waterfront_, _yr_renovated_ and _view_ the variables (i.e. columns) are complete in the sense that no missing values appear.<br>
But, additionally let us check wether there are any duplications or multiple entries which need to be cleaned.

In [ ]:
df["id"].duplicated().value_counts()

Indeed, the _id_ which is unique for each house has 177 duplications. So, are there really duplications of complete rows or does the house id just occur more than once? (For example indicating that a house has been bought and sold several times within the given time period) 

In [ ]:
df.duplicated().unique()

As there is not a single TRUE value, there are no duplications of complete rows. So, for now we will keep them. <br>
Lastly, we will take a brief view on some basic descriptive statistical parameters for each variable:

In [ ]:
df.describe()

Here, allthough being numeric, statistics for variables such as _id_, _waterfront_, _zipcode_, _latitude_ and _longitude_ can be ignored.  
To get more insights, let's visualise some variables of the table which might be of special interest for our purpose (i.e. the requests of the stakeholder):
- the _price_
- the _year built_ (and _year renovated_ if applicable)
- the _living size_ and overall _(lot) size_ of the houses
- quality - specified via _condition_ and _grade_

In [ ]:
fig, axes = plt.subplots(1, 7, figsize=(18, 8))

sns.boxplot(ax=axes[0], data=df.price)
sns.boxplot(ax=axes[1], data=df.yr_built)
sns.boxplot(ax=axes[2], data=df['yr_renovated'])
sns.boxplot(ax=axes[3], data=df['sqft_living'])
sns.boxplot(ax=axes[4], data=df['sqft_lot'])
sns.boxplot(ax=axes[5], data=df['condition'])
sns.boxplot(ax=axes[6], data=df['grade'])

axes[0].set_title('Price [USD]')
axes[1].set_title('Year Built')
axes[2].set_title('Year Renovated')
axes[3].set_title('size living [sqft]')
axes[4].set_title('size lot [sqft]')
axes[5].set_title('condition')
axes[6].set_title('grade');

**First insights regarding the overall data:** <br>
- we can see that variables _price_, _sqft_living_ and _sqft_lot_ appear to be right skewed distributed as these have a couple of outliers towards higher values 
- _yr_built_ and _condition_ seem to have a rather symmetrical distribution
- for most of the houses, the _grade_ varies between 6 and 9 with a few outliers towards both sides of the distribution 
- the variable _yr_renovated_ seems to be corrupted. So lets have a look at the values:

In [ ]:
df.yr_renovated.unique()

So, there are both nan and zeros which we need to have in mind when proceeding with the data cleaning. We already know that there are 3842 nan, so let's count the zeros, as well:

In [ ]:
(df['yr_renovated'] == 0).sum()


So, out of 21,597 values we have 3,842 nan plus 17,011 zeros, ergo - at maximum - 744 values left considering there may still be some duplications.<br>
Note that the zeros can likely be interpreted as "no", so these houses have not been renovated at all:
- houses renovated (ca. 744)
- houses not renovated (ca. 17,011)
- no statement (ca. 3,842)

**First insight regarding a (possible) geographical pattern:** <br>
Last, lets have a look if the year a house was built relates to a certain area, e.g. different development areas for different decades.

In [ ]:
sns.scatterplot(data=df, x='long', y='lat', hue='yr_built'); 

We can clearly see the western concentration of the population and a lot of newer houses (built in 2000) close to the coastline, especially in the north. Besides that, in the very west there seems to be an island with mainly older houses.

## 2) Initial Hypotheses about the Dataset

Hypotheses related to the stakeholder: <br>
To buy a central neighborhood house in middle price range with right timing (within a year) it will
- H1) be older than 50 years and has not been renovated in the last 25 years or
- H2) has a below-average grade or
- H3) has a below-average livingsquare

## 3) Explore and clean the dataset



Before we start with the cleaning of the data, it is suitable to specify and define the stakeholders requests in terms of available parameters:<br>
a) lively, central neighborhood
b) middle price range
c) right timing (within a year)

- a) lively, central neighborhood <br>
The term "lively" is a bit vague and can be interpreted differently. So for a non-local it cannot be linked without domain knowledge to certain longitudes/latitudes or zip codes. The term "central neighborhood" on the other hand can be addressed directly to a - yet to be defined - subset of zip codes. One may either define "central" in geographic terms or "central neighborhood" in terms of an area with higher population density.<br>

- b) middle price range <br>
This is a rather concrete condition. But instead of using the price itself, we will address it by normalizing the price by estimating the price per squarefeet. Furthermore, we define "middle price range" by choosing the 40% to 60% percentiles.

- c) right timing (within a year) <br>
So, our stakeholder wants to time the equity market. For that purpose, we will generate a new variable called "month" to analyze if there is a month showing significant lower house prices than other months.

### Add/remove columns

In [ ]:
# We create a copy of the original dataset and will adopt that one for our aims.
df2 = df.copy()

# As we want to know the right timing of buying within a year, we generate a 'month' column
df2['month'] = df2.date.dt.month

# Add prices for lot and living as prices per squarefeet
df2['price_lot_per_sqft'] =  df2['price'] / df2['sqft_lot']
df2['price_living_per_sqft'] =  df2['price'] / df2['sqft_living']

# For our purpose, we don't need sqft_above and sqft_basement, as these add up to sqft_living and we are only interested 
# in the overall size of the living area and overall lot size.
df2.drop(['sqft_above','sqft_basement'], axis=1, inplace=True)

# As the stakeholder set no further conditions relating to the interior of the house, we will skip variables such as "waterfront", 
# number of floors, bathrooms, bedrooms etc.
df2.drop(['bedrooms','bathrooms','floors','waterfront','view'], axis=1, inplace=True)

In [ ]:
df2

### Filtering

Living central in King County basically means living in Seattle. Therefore, we seek to select all zip codes in that county that belong to the city of Seattle. Luckily, we can find [here](https://www.usmapguide.com/washington/seattle-zip-code-map/) a list which we will use.  [This map](https://statisticalatlas.com/county/Washington/King-County/Population) showing the distribution of population density in King County supports the argument to use Seattle zip codes only.

In [ ]:
# zip codes in Seattle
zc_seattle = [98101 ,98102 ,98103 ,98104 ,98105 ,98106 ,98107 ,98108 ,98109 ,98110 ,98111 ,98112 ,98114 ,98115 ,98116 ,98117 ,98118 ,98119 ,98121 ,98122 ,98124 ,98125 ,98126 ,98129 ,98131 ,98132 ,98133 ,98134 ,98136 ,98138 ,98144 ,98145 ,98146 ,98148 ,98151 ,98154 ,98155 ,98158 ,98160 ,98161 ,98164 ,98166 ,98168 ,98170 ,98171 ,98174 ,98177 ,98178 ,98181 ,98184 ,98185 ,98188 ,98190 ,98191 ,98195 ,98198 ,98199]

In [ ]:
# Filter only houses in Seattle
df2 = df2[df2['zipcode'].isin(zc_seattle)]
df2.shape

We still have 8,973 buy/sell transactions to work with. Counting the transactions where we don't know if the house has been renovated or not yields 1,646. Let us remove them, too.

In [ ]:
df2.isna().sum()

In [ ]:
df2 = df2.dropna()

### Testing the hypotheses

#### H1) be older than 50 years and has not been renovated in the last 25 years

In [ ]:
# Estimating the "middle price range" for prices living per squarefeet
living40,living60 = df2.price_living_per_sqft.quantile([0.40, 0.60])

To test the first hypothesis, we plot the histogram for the living price per square feet and estimate the age of the houses within our price range.

In [ ]:
sns.histplot(data=df2, x="price_living_per_sqft")
plt.plot([living40, living40], [0, 420], color='r')
plt.plot([living60, living60], [0, 420], color='r')
plt.title('Histogram of price per sqft for living area' + '\n' + '(red lines indicate 40 to 60 percentile range)');

In [ ]:
# generate a mask to select only mid-price range
mask1 = (living40 <= df2['price_living_per_sqft']) & (df2['price_living_per_sqft'] <= living60)

In [ ]:
# plot age of houses for selected price range
sns.histplot(data=df2[mask1], x="yr_built", bins=30)
plt.plot([1972, 1972], [0, 140], color='r')
plt.title('Histogram of the house age for mid-price segment'+ '\n' + '(houses right of red line are younger than 50 years old)');

We can clearly see that there are houses younger than 50 years. So, at his point we can already say that **hypothesis H1 is wrong**. <br>
But still, let us have a look if the houses older than 50 years have all been renovated or not in the last 25 years.

In [ ]:
# generate a mask to select only mid-price range older than 50 years
mask2 = (living40 <= df2['price_living_per_sqft']) & (df2['price_living_per_sqft'] <= living60) & (df2['yr_built']<=1972)

In [ ]:
# plot histogram of the renovation year for mid-priced houses older than 50 year
sns.histplot(data=df2[mask2], x="yr_renovated", bins=30)
plt.title('Histogram of the renovation year of houses for mid-price segment'+ '\n' + 'and being older than 50 years');

So, the vast majority at 0 show that most of the old houses have not been renovated at all, but let's zoom into the last decades:

In [ ]:
# generate a mask to select only mid-price range older than 50 years & have been renovated
mask3 = (living40 <= df2['price_living_per_sqft']) & (df2['price_living_per_sqft'] <= living60) & (df2['yr_built']<=1972) & (df2['yr_renovated']>1)

# plot histogram of the renovation year for mid-priced houses older than 50 year
sns.histplot(data=df2[mask3], x="yr_renovated", bins=30)
plt.plot([1997, 1997], [0, 17], color='r')
plt.title('Histogram of the renovation year of old houses for mid-price segment'+ '\n' + '(red line indicates 25 years renovation threshold)');

As we can see, even if we had only houses older than 50 years fitting to the conditions of the stakeholder (which is not the case), there are houses that have been renovated within the last 25 years, hence, **hypothesis H1 is wrong**.

#### H2) has a below-average grade

First let's visualise the grade distributions and calculate the overall mean grade for houses in Seattle, i.e. the center. Its a measure based on the King County grading system (between 1 - 13) and we will compare it with the data having constrains:

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=df2['grade'], name='center grade'))
fig.add_trace(go.Histogram(x=df2[mask1].grade, name='center grade with mid-price constrains'))

# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)

fig.update_layout(
    title_text='Histograms of the grade distributions', 
    xaxis_title_text='grade', 
    yaxis_title_text='count') 

fig.show()

In [ ]:
df2['grade'].mean()

In [ ]:
df2[mask1].grade.mean()

The numbers show that the grade for the houses does slightly increase using the stakeholders constrains compared to the overall grade. Therefore, **hypothesis H2 has been proven wrong**, as well.

#### H3) has a below-average livingsquare

Similar to hypothesis 2, we will calculate the average livingsquare for the center and compare it with the average livingsquare only in the 40% to 60% percentile range.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=df2['sqft_living'], name='center living size in sqft'))
fig.add_trace(go.Histogram(x=df2[mask1].sqft_living, name='center living size in sqft with mid-price constrains'))

# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)

fig.update_layout(
    title_text='Histograms of the living sizes in sqft distributions', 
    xaxis_title_text='living size in sqft', 
    yaxis_title_text='count') 

fig.show()

In [ ]:
df2['sqft_living'].mean()

In [ ]:
df2[mask1].sqft_living.mean()

Again, the average slightly increases. So, the average size of the living area inside the house is a tiny bit higher applying the mid-range price constrain. Therefore, **hypothesis H3 has also been proven wrong**.

#### Summary of hypotheses

All three hypotheses that have been stated in the beginning have been proven to be wrong. Although these were not very provocative and seemed to be "no brainer", the examinations show an unexpected result emphasizing the need to look into the numbers.

### Market timing

To estimate the best time for buying a house, we will group the transactions in the center in the mid-price range by month:

In [ ]:
df3 = df2[mask1].groupby(['month']).median()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

fig.suptitle('Median prices for living and lot per sqft over the year', fontsize=24)
sns.lineplot(ax=axes[0], data=df3["price_living_per_sqft"])
sns.lineplot(ax=axes[1], data=df3["price_lot_per_sqft"]);

Here, we provide two possible answers for our stakeholder. Do we want to maximize the living area, the best choice is probably to buy in october. Do we concern about a big lot spring seems to be the right time - so in march or april.

## 4) Recommendations for the stakeholder

Given the stakeholders constrains, we might suggest:
- an overall reasonable price in the mid-price segment for living area per squarefeet is between 263 USD and 321 USD
- avoid buying in january and june and look for houses in october for maximizing living area
- to maximize lot size, buy in march or april
- the living area should have a size of about 1,800 squarefeet
- the grade should at least be 7
- most houses have been built 1940 - 1960 and 2000 - 2015. So when age is a criterium, looking for houses in these built phases enhances the supply